In [ ]:
import os

import pandas as pd
import numpy as np

from smrt import make_snowpack, make_model, sensor_list, make_soil, SMRTError

from smrt.inputs.make_medium import make_medium

from mw_antarctica.frequency_colors import frequency_colors


import scipy.stats

%load_ext autoreload
%autoreload 2

%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
sp_xls = pd.ExcelFile("data/CanadaData/SP.xlsx")

# each sheet is one snowpit
sps = sp_xls.sheet_names

sps = [sp for sp in sp_xls.sheet_names if not sp.startswith("CB20")]

len(sps)

In [ ]:

def prepare_snowpit(sp):

    profile = sp_xls.parse(sp)

    temp_substrate = min(profile['TempSol'].iloc[0], 273.15)

    profile['z'] = profile['Hneige'] * 0.01

    profile = profile.drop(columns=['TempSol', 'TEMPERATURE', 'IRIS', 'Hneige', 'Hneige_cm', 'Ropt', 'Hneige (m)']).dropna()

    profile = profile.rename(columns={'Temp_Kel': 'temperature',
                                      'Density': 'density'})

    return temp_substrate, profile
        
snowpits = {sp: prepare_snowpit(sp) for sp in sps}



In [ ]:
freqs = [10, 19, 37, 89]


def run_model(temp_substrate, profile, microstructure):
    density_ice = 917.
    porod_length = 4 * (1 - profile.density / density_ice) / (density_ice * profile.SSA)
    
    #UTS 1.4749999999999983 21.449086018284927
    #USEXP 1.274999999999999 21.68267536399686
    #USHS 3.1749999999999923 20.416629004819455
    
    K_TS = np.where(profile.SSA > 15, 0.6, 1.47)
    #coef_GF = np.where(profile.SSA > 15, 5.5, coef)
    K_SEXP = np.where(profile.SSA > 15, 0.625, 1.27)
    K_SHS = np.where(profile.SSA > 15, 0.64, 2.35)
        
    microstructure_args = {
        "USEXP": dict(microstructure_model="unified_scaled_exponential", porod_length=porod_length, polydispersity=K_SEXP),
        "UTS": dict(microstructure_model="unified_teubner_strey", porod_length=porod_length, polydispersity=K_TS),
        "USHS": dict(microstructure_model="unified_sticky_hard_spheres", porod_length=porod_length, polydispersity=K_SHS)
    }

    substrate = make_soil("soil_wegmuller", permittivity_model="montpetit2008",
                          temperature=temp_substrate, roughness_rms=1.93e-3)

    sp = {m: make_medium(profile, **microstructure_args[m], substrate=substrate) for m in microstructure}

    model = make_model("iba", "dort", rtsolver_options=dict(prune_deep_snowpack=8, error_handling='nan'))
    sensor = sensor_list.amsr2(map(str, freqs))

    results = model.run(sensor, sp, parallel_computation=False)

    results = results.to_dataframe()
    results = results.unstack()
    results.index = ["_".join(ind) for ind in results.index]  # collapse multiindex by joining names with _

    return results

polydispersity_DH = 1.32
results = dict()
microstructure = ['UTS', 'USEXP', 'USHS']

for sp in snowpits:
    if snowpits[sp] is None:
        continue
    res = run_model(*snowpits[sp], microstructure)

    if res is None:
        continue
    results[sp] = res

results = pd.DataFrame(results).T

In [ ]:
sp

In [ ]:
obs = pd.read_csv("data/CanadaData/TB.csv")

In [ ]:
database_with_result = results.join(obs.set_index("CorresSP"))

In [ ]:
# Residual

residual = pd.DataFrame([(m, freq, pola, f'{freq}{pola}', database_with_result[f'{freq}{pola} MES'] - database_with_result[f'{freq}{pola}_{m}']) \
                         for m in microstructure for freq in freqs for pola in 'HV'],
                        columns=('microstructure', 'freq', 'pola', 'channel', 'residual'))
    

def rmse_func(x):
    return np.sqrt(np.mean(x**2))
    
def bias_func(x):
    return np.mean(x)
    
residual['RMSE'] = residual['residual'].apply(rmse_func)
residual['bias'] = residual['residual'].apply(bias_func)
#residual['std'] = residual['residual'].apply(np.std)

residual['RMSE']

residual

# compare RMSE with obs.
stats_pola = residual.groupby(['microstructure', 'freq', 'pola']).aggregate({'RMSE': rmse_func, 'bias': bias_func}) #, 'std': rmse_func })
stats = residual.groupby(['microstructure', 'freq']).aggregate({'RMSE': rmse_func, 'bias': bias_func}) #, 'std': rmse_func })

#np.sqrt((stats_pola.loc['UTS', 19, 'V'] **2 + stats_pola.loc['UTS', 37, 'V']**2)/2)

#stats
stats.groupby('microstructure')['RMSE'].apply(rmse_func)
#stats.groupby('microstructure')['bias'].apply(rmse_func)



In [ ]:
freq=89
pola='H'
m='UTS'

def ttest_func(freq, pola, m):
    return scipy.stats.ttest_1samp(pd.concat(database_with_result[f'{freq}{pola} MES'] - database_with_result[f'{freq}{pola}_{m}'] for pola in 'HV').dropna(), 0)
    
    
pd.DataFrame([(m, freq, *ttest_func(freq, pola, m)) for m in microstructure for freq in freqs],
                        columns=('microstructure', 'freq', 't', 'p'))

In [ ]:
freqs = [10, 19, 37, 89]

def plot_all():
    f, axs = plt.subplots(4, 1, figsize=(11, 7), 
                          sharex=True,
                          gridspec_kw=dict(top=0.98, bottom=0.13, hspace=0.06, wspace=0.08),
                          )
    f.canvas.layout.height = '1000px'
    
    simulations = {
                    #'USHS': ('_USHS', dict(marker='x', linestyle='--', alpha=0.9)),
                    #'USEXP': ('_USEXP', dict(marker='x', linestyle='--', alpha=0.9)),
                    'UTS': ('_UTS', dict(marker='x', linestyle='--', alpha=0.9)),
                    #'GF': ('_GF', dict(marker='s', linestyle='--', alpha=0.7)),
                    'Obs.': (' MES', dict(marker='o', alpha=0.5)),
                 }

    #color_V = ("#5E9732", "#277455")
    #color_H = ("#AA5B39", "#993350")

    #color_V = ('#80E135', '#35E69F')
    #color_H = ('#FF753A', '#F7396F')

    color_V = ('#2F71A3', '#4D5CE7')
    color_H = ('#FEC93C', '#FEAB3C')
 
    ylabel = "Brightness\ntemperature (K)"
    for i in [0, 1, 2, 3]:
        #axs[i][0].set_ylabel(ylabel)
        axs[i].set_ylabel(ylabel)

    #axs = {freq: axs[i // 2][i % 2] for i, freq in enumerate(freqs)}
    axs = {freq: axs[i] for i, freq in enumerate(freqs)}
    
    database_with_result_newindex = database_with_result.reset_index().set_index((i for i in range(1, len(database_with_result) + 1)))
    database_with_result_newindex = database_with_result.reset_index().set_index(np.arange(1, len(database_with_result) + 1))    

    for i, freq in enumerate(freqs):
        for sim in simulations:
            suffix, style = simulations[sim]
         
            #icolor = 0 if suffix == '' else 1
            icolor = 1
       
            v = database_with_result_newindex['%iV%s' % (freq, suffix)]
            v = v.where(np.isfinite(database_with_result_newindex['%iV MES' % freq]))
            axs[freq].plot(v.index, v, **style, lw=1, color=color_V[icolor])
            h = database_with_result_newindex['%iH%s' % (freq, suffix)]
            h = h.where(np.isfinite(database_with_result_newindex['%iH MES' % freq]))
            axs[freq].plot(h.index, h, **style, lw=1, color=color_H[icolor])
            
            if sim != 'Obs.':
                rmse = stats.loc[(sim, freq)]['RMSE']
                rmseV = stats_pola.loc[(sim, freq, 'V')]['RMSE']
                rmseH = stats_pola.loc[(sim, freq, 'H')]['RMSE']
                bias = stats.loc[(sim, freq)]['bias']
                biasV = stats_pola.loc[(sim, freq, 'V')]['bias']
                biasH = stats_pola.loc[(sim, freq, 'H')]['bias']
                print(rmse)
                kwargs = dict(xycoords=('axes fraction', 'axes fraction'), alpha=0.8, fontsize=9)
                axs[freq].annotate(f'RMSE({freq}):   {rmse:.1f}K', xy=(0.05, 0.19), **kwargs)
                axs[freq].annotate(f'RMSE({freq}V): {rmseV:.1f}K', xy=(0.05, 0.11), **kwargs)
                axs[freq].annotate(f'RMSE({freq}H): {rmseH:.1f}K', xy=(0.05, 0.03), **kwargs)
                axs[freq].annotate(f'bias({freq}):   {bias:.1f}K', xy=(0.19, 0.19), **kwargs)
                axs[freq].annotate(f'bias({freq}V): {biasV:.1f}K', xy=(0.19, 0.11), **kwargs)
                axs[freq].annotate(f'bias({freq}H): {biasH:.1f}K', xy=(0.19, 0.03), **kwargs)
            
                #std = stats.loc[(sim, freq)]['std']
            

        #axs[freq].xaxis.set_tick_params(rotation=90, labelsize=6)
        #axs[freq].xaxis.set_tick_params(labelsize=6)
        #if i < 2:
        #    axs[freq].xaxis.set_ticklabels([])
    
        axs[freq].set_title("%i GHz" % freq, y=0.05)
        axs[freq].set_ylim((100, 275))
        axs[freq].grid(alpha=0.2)

    plt.xlim((0, len(database_with_result_newindex)))
    plt.tight_layout()
    plt.savefig("fig-simulation-unified-arctic.pdf")
    plt.xlabel("Site number")
plot_all()

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(4, 7), sharex=True)


def plot(database_with_result, microstructure, ax, where):
    
    ax.plot([125, 260], [125, 260], '0.8')
                
    for freq in [10, 19, 37, 89]:
        for pola in 'HV':
            channel = f"{freq}{pola}"
            symbol = "v" if pola == 'H' else "o"
            
            if where == 'Arctic':
                mes = database_with_result[channel+" MES"]
            elif where == 'Antarctic':
                if False and freq == 89:
                    mes = database_with_result[channel+"_summer"]
                else:
                    mes = database_with_result[channel]
            
            label = f'{freq} GHz' if pola == 'V' else None
            ax.plot(mes,
                     database_with_result[channel+"_"+microstructure], 
                     symbol, color=frequency_colors[f"{freq:02}"], alpha=0.4, markersize=4, label=label)
    
major_ticks = range(120, 261, 20)
for ax in axs:
    ax.grid(alpha=0.2)
    ax.axis('equal') #set_aspect('equal', 'box')
    ax.set_xticks(major_ticks)
    ax.set_yticks(major_ticks)
    
plot(pd.read_csv("results/sites-database-with-results-unified-antarctica.csv"), "USHS", axs[0], where='Antarctic')
plot(database_with_result, "UTS", axs[1], where='Arctic')


#axs[0].xaxis.set_ticklabels([])
axs[0].set_title('Antarctic', y=0.9)
axs[1].set_title('Arctic', y=0.9)
axs[0].legend()
axs[1].legend()


for i in [0, 1]:

    axs[i].set_ylabel("Modeled brightness temperature (K)")
axs[1].set_xlabel("Observed brightness temperature (K)")    

f.tight_layout()

plt.savefig("fig-simulation-both-scatterplot.pdf")